In [ ]:
# example_srs_sim.py

import numpy as np
import tensorflow as tf
from sionna.nr import CarrierConfig
from sionna.nr.srs_config import SRSConfig
from sionna.nr.srs_pilot_pattern import SRSPilotPattern
from sionna.ofdm import ResourceGrid

def main():
    # 1) Create a CarrierConfig.
    carrier = CarrierConfig()
    carrier.n_size_grid = 24           # e.g., 24 RBs
    carrier.subcarrier_spacing = 15    # in kHz
    carrier.cyclic_prefix = "normal"   # ensures 14 OFDM symbols per slot
    carrier.slot_number = 10
    carrier.frame_number = 0

    # For debugging, print number of symbols per slot:
    print("Carrier num_symbols_per_slot =", carrier.num_symbols_per_slot)

    # 2) Create two SRSConfig objects.
    # Transmitter 1: 2 ports, SRS occupies 2 symbols.
    srs1 = SRSConfig(carrier_config=carrier)
    srs1.num_srs_ports = 2
    srs1.symbol_start = 10
    srs1.num_srs_symbols = 2
    srs1.c_srs = 5
    srs1.b_srs = 0
    srs1.k_tc = 2
    srs1.srs_period = [2, 1]  # period = 2 slots, offset = 1
    srs1.n_srs_id = 123
    srs1.cyclic_shift = 0
    srs1.cyclic_shift_hopping = True
    srs1.cyclic_shift_hopping_id = 1
    srs1.enable_eight_port_tdm = False

    # Transmitter 2: 1 port, SRS occupies 1 symbol.
    srs2 = SRSConfig(carrier_config=carrier)
    srs2.num_srs_ports = 1
    srs2.symbol_start = 11
    srs2.num_srs_symbols = 1
    srs2.c_srs = 3
    srs2.b_srs = 1
    srs2.k_tc = 2
    srs2.srs_period = "on"  # always on
    srs2.n_srs_id = 456
    srs2.cyclic_shift = 0
    srs2.cyclic_shift_hopping = False
    srs2.enable_eight_port_tdm = False

    # 3) Build a pilot pattern for these SRS configurations.
    pilot_pattern = SRSPilotPattern([srs1, srs2])
    print("Pilot pattern created. Mask shape =", pilot_pattern.mask.shape)
    print("Pilot pattern pilots shape =", pilot_pattern.pilots.shape)

    # 4) Create a ResourceGrid (Sionna's OFDM grid) and apply the pilot pattern.
    num_tx = pilot_pattern.mask.shape[0]
    num_streams_per_tx = pilot_pattern.mask.shape[1]  # equals max ports across configs
    rg = ResourceGrid(
        num_ofdm_symbols=carrier.num_symbols_per_slot,
        fft_size=carrier.n_size_grid * 12,
        num_tx=num_tx,
        num_streams_per_tx=num_streams_per_tx,
        pilot_pattern=pilot_pattern,
        cyclic_prefix_length=carrier.cyclic_prefix_length
    )
    x_tx = rg.apply_pilot_pattern(None)
    print("Resource grid shape after applying pilot pattern:", x_tx.shape)

    # 5) Print additional SRS info.
    print("SRS info for srs1:", srs1.info)
    print("SRS info for srs2:", srs2.info)

if __name__ == "__main__":
    main()
